# The goal of this program is to create a data frame that includes:
## Champion picks of every current Challenger player (including their climb games) **from season start**

In [1]:
import pandas as pd
import os
import cassiopeia as cass
import requests
import random
from ratelimit import limits
import json
import datetime
pwd = os.getcwd()

cass.set_riot_api_key("READCTED")

key = 'READCTED'

### Getting used to using Champion IDs & Returning Values into Arrrays

In [2]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region="NA")

summoner_name = []

for item in challenger_league:
    summoner_name.append(item.summoner.name)

print(summoner_name)


Making call: https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
['Ryuzu Shima', 'Doxa', 'Anotherhope', 'sean twitch god', 'Bon Garçon', 'ScaryJerry', 'MONSTER CHAD GOD', 'Berik', 'TTV LegitKorea', 'rallyracer21', 'Eyla', '767602', 'Echídna', 'nifty jg', 'bao bu dong', 'An Jıng', 'Dacryagogue', 'picapica', 'Smoothie', 'Latence1', 'TL Hans samaa', 'Kind Jungle', 'IMT PoE', 'Josedeodo', 'dtro18', 'Le Kachu', 'Revenge', 'TTV KoKooPuffs', 'arrowIoI', 'Bot0511', 'Prymari', 'Ssumdayday', 'tomeito', 'Aki Haruka', 'khaii', 'MoeTube', 'Competition9', 'Binkx123', 'Bmfx', 'Doubtfull', 'FORWARDFORWARDFO', 'Dao Dwen Dwen', 'mohanno', 'TwtchTV Sheiden', 'TFBlade2', 'AnDa', 'soupfan4', 'Icelandic Hero', 'Frenq', 'yerrrrrikan', 'QuinnAD Daddy', 'hotazy', 'Mr Bluehat', 'bryan3838', 'Oronuke', 'Shiba lnu', 'PuTongDaYe', 'Yung Fappy', 'warcyclone', 'Sayarin', 'Aiden Yasuo', 'rockonhi5', '21 Stylish', 'rehcra', 'Draven baka', 'GOOD LUCK SOL', 'Szochs', 'Jauny12', 'twtv C

In [3]:
summoner_puuid = []  

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    summoner_puuid.append(bucket.puuid)
    
summoner_puuid


Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/rcQEuh1tMcYkoz5ZWS4E16b9kjHTfql-QevZLl2bWrnZgsQ
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/zlfC9_0GAfHNCR7ejX8U_XPdv--SmL8GAL4enp7mYXM3N7s
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/fozc9oPIpVybHjtejfzDRmuTipMYAITb1zQWVzvT7628kAE
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/97Xh01a982o3vfQq8T7VlU0LGoAMp_ydsAttLbBEv57bBUU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/5JJsS2PSG6xIaWqeVp6rFEcLt0SK7v7B0DRlLaLY436y6z2r
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/nwd5SwSP-68hTG8zjlcjyADywZ4s88VCRo-TSeQBUxzZsmU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/vee8rIBV6lYKBcnXVyNEUvKoDrB1d-7yNyulJDgNGFcOWK8
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/-UQbcDlcMUnFbeNkwqyy0WAsWS2Cwlx_uoICPogrHzzP2MM
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summ

['bZqmWNtUY0jKOlnOgSM8R4msEcw-2hvilnBvmA-EIbWQtbsc6ihGFTxTHRWIxT8_kNBopbm1N9dJvg',
 '2QBzTHAEe3g_ssuHqfN1ihqADOxLGFI4v4l7L3sWM1FtrOgTfK5sdV3t6CZPtdmbf0LW97yW0okd1g',
 'A7THoT1EUaXiD8fzaGhilJPzK-0Yr6HgGAwE1rrT5OEhUlmey4yzkrcnJLY6E39N8WWPhMRjNI9kKw',
 '-EP2oQHedhm50h7ojEVRGvHK-n3wjReMIogCRcB-gOYqBHapz1SVmL5lYVvJDj6OBtwbQ5akAMTK9Q',
 'VsENhWtbOPo-zyLVJIX4Buv5oQJcT8PAMl04d93Irufn8RWy_KcuJHE3ikvkv2O_DdDJiw8wy92tow',
 'L5H8BGKQ_05acfr3CQt1vtjbue5yvIyUiCLsJiRs3ftOn_gINAK0sHxRGweVQpHubLZAX9vC-V01Sg',
 'Nml2Jc0wBxg8R3VsTjsgGXm-dkgWcuY7QDbD4cGQp3mpQBplGLEJ0Zqr2pBjClImNNAP3pmmNoqJ9g',
 'vxNdOVPsoQ-xD1lKRnXUoS1e6upouSSLsv4J_FAIjHCpGiErQJB1QDMeq1hfLY1NPFXAYYCtkM4wvA',
 'jkTPeZGg7GFSAtjEwZNt7ZnYNR0-Q7zYpUm_zoEQauSCuIFP91-vSawLVhus7QQ-rrtTMBT7Nc4rlw',
 'j77lOzlIrzT1vvxtcLILYmscLOYLbtTf6T4iBOeWfO2XOGk54C9H4iDsXQImUXNBq4d8_TRs_oWKvg',
 'sW_JBbGPU8Qj-cQ-z-uK7xdqvzhjJMxakQXIlpNQWa3ulQ2rkPJszq8p6xMieUGppH8uElopnoW1Pw',
 'aCnc11AAUEr2y8Z1B4ntzzQloJwVdszV7FaHxLu3JsYQIGv5Q_qG5-TdcBqP3aUjP8Bsl_CWxzlCbw',
 'JS

## Now that I have a list of summoner names and puuids, I can start pulling match histories

In [4]:
steve_history = []

steve = cass.Summoner(name='YoRHa Destiny', region='NA')
steve_history.append(cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=steve.puuid,
                                             continent=cass.data.Continent.americas))


         

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/YoRHa%20Destiny


In [5]:
print(steve)

steve_history[0][0].id

Summoner(id=cSFn_m6aqcr2x-oW1iC_TAuxUXUJKXNFylk_H1rGq6V9Euc, account_id=nGRaUbLW0jC1G5rBuctOfkQU-Z3vW3HaDCtgv9Y8owVe9g, name='YoRHa Destiny', puuid='9h-joR6pWHaTzM9wXXK8IV_nUU4GwM36C0lUvi7NSMw8HjPD1y3lGKrxiJqxmLCeH7X0HeGBF03dcQ')
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/9h-joR6pWHaTzM9wXXK8IV_nUU4GwM36C0lUvi7NSMw8HjPD1y3lGKrxiJqxmLCeH7X0HeGBF03dcQ/ids?beginIndex=0&endIndex=100&queue=420


'NA1_4223102929'

In [6]:


steve_champ = []

for item in steve_history[0]:
    steve_champ.append(item.participants[steve].champion.name)
    

steve_champ



Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4223102929
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.4.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4223087515
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222778607
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222786977
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222755547
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222751834
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222708062
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222702225
Making call: https://americas.a

['Tristana',
 'Tristana',
 'Tristana',
 'Tristana',
 'Tristana',
 'Jinx',
 'Jinx',
 'Tristana',
 'Tristana',
 'Tristana',
 'Jinx',
 'Jinx',
 'Jinx',
 'Draven',
 'Jinx',
 'Tristana',
 'Tristana',
 'Jinx',
 'Jinx',
 'Jinx']

## Now that I kinda get how to pull match histories, I need to figure out how to store this list for all challenger players 

In [7]:
total_match_list = []

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    challenger_history = (cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=bucket.puuid,
                                             continent=cass.data.Continent.americas))
    total_match_list.append(challenger_history)


total_match_list

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/bZqmWNtUY0jKOlnOgSM8R4msEcw-2hvilnBvmA-EIbWQtbsc6ihGFTxTHRWIxT8_kNBopbm1N9dJvg/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/2QBzTHAEe3g_ssuHqfN1ihqADOxLGFI4v4l7L3sWM1FtrOgTfK5sdV3t6CZPtdmbf0LW97yW0okd1g/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/A7THoT1EUaXiD8fzaGhilJPzK-0Yr6HgGAwE1rrT5OEhUlmey4yzkrcnJLY6E39N8WWPhMRjNI9kKw/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/-EP2oQHedhm50h7ojEVRGvHK-n3wjReMIogCRcB-gOYqBHapz1SVmL5lYVvJDj6OBtwbQ5akAMTK9Q/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/VsENhWtbOPo-zyLVJIX4Buv5oQJcT8PAMl04d93Irufn8RWy_KcuJHE3ikvkv2O_DdDJiw8wy92tow/ids?beginIndex=0&endIndex=100&queue=420
Making call: ht

[[<cassiopeia.core.match.Match at 0x1c1641a4970>,

In [8]:

total_match_list[0]

In [9]:

champion_picks = []
data = []
matches = []

for position, match_list in enumerate(total_match_list):
    bucket = cass.Summoner(name=summoner_name[position], region='NA')
    for match in match_list:
        champion = match.participants[bucket].champion.name
        champion_picks.append(champion)
        matches.append(match)
        tup = (bucket.name, champion, match)
        data.append(tup)

champion_picks
data        
matches

        


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221997016
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221983107
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221172975
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221082210
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221076268
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4220883536
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4220806671
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4220781808
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4220560086
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4219849020
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4219763700
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_421

 ...]

In [10]:
data.sort

data

[('Ryuzu Shima',
  'Renata Glasc',
  <cassiopeia.core.match.Match at 0x1c1641a4970>),
 ('Ryuzu Shima',
  'Renata Glasc',
  <cassiopeia.core.match.Match at 0x1c1641a8820>),
 ('Ryuzu Shima', 'Pyke', <cassiopeia.core.match.Match at 0x1c1641a88b0>),
 ('Ryuzu Shima',
  'Renata Glasc',
  <cassiopeia.core.match.Match at 0x1c1641a8640>),
 ('Ryuzu Shima',
  'Renata Glasc',
  <cassiopeia.core.match.Match at 0x1c1641a8310>),
 ('Ryuzu Shima', 'Yuumi', <cassiopeia.core.match.Match at 0x1c1641a87c0>),
 ('Ryuzu Shima', 'Rell', <cassiopeia.core.match.Match at 0x1c1641a8970>),
 ('Ryuzu Shima', 'Rakan', <cassiopeia.core.match.Match at 0x1c1641a8a30>),
 ('Ryuzu Shima', 'Nautilus', <cassiopeia.core.match.Match at 0x1c1641a8af0>),
 ('Ryuzu Shima', 'Braum', <cassiopeia.core.match.Match at 0x1c1641a8bb0>),
 ('Ryuzu Shima', 'Rakan', <cassiopeia.core.match.Match at 0x1c1641a8c70>),
 ('Ryuzu Shima', 'Rakan', <cassiopeia.core.match.Match at 0x1c1641a8d30>),
 ('Ryuzu Shima', 'Seraphine', <cassiopeia.core.match.Ma

In [11]:
dataframe = pd.DataFrame(data, columns= ['Summoner', 'Champion', 'Match'])

dataframe

,Summoner,Champion,Match
0,Ryuzu Shima,Renata Glasc,"Match(id=4221997016, region='AMERICAS')"
1,Ryuzu Shima,Renata Glasc,"Match(id=4221983107, region='AMERICAS')"
2,Ryuzu Shima,Pyke,"Match(id=4221172975, region='AMERICAS')"
3,Ryuzu Shima,Renata Glasc,"Match(id=4221082210, region='AMERICAS')"
4,Ryuzu Shima,Renata Glasc,"Match(id=4221076268, region='AMERICAS')"
...,...,...,...
5995,FLY Diamond,Leona,"Match(id=4204384386, region='AMERICAS')"
5996,FLY Diamond,Leona,"Match(id=4204319650, region='AMERICAS')"
5997,FLY Diamond,Thresh,"Match(id=4204332563, region='AMERICAS')"
5998,FLY Diamond,Jhin,"Match(id=4204060896, region='AMERICAS')"


## I now know how pull matchlists, sort them by summoner, and store them into a data frame

### Now I need to figure out how to see which champions have the most picks, and test the things like:

#### "Most one trick", "Least one trick", "Highest Pick Rate for Climb", "Lowest Pick Rate for Climb" etc...

In [12]:
dataframe

dataframe_modified = dataframe.copy()

matches_set = set(matches)

match_count = len(matches_set)

match_count


2895

In [13]:
dataframe_modified.describe()

,Summoner,Champion,Match
count,6000,6000,6000
unique,300,154,2895
top,Ryuzu Shima,Jinx,"Match(id=4220618783, region='AMERICAS')"
freq,20,203,10


# Most Picked Champions

In [14]:
from collections import Counter

In [15]:
champ_pick_num = Counter(elem[1] for elem in data)

sorted_10  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=True)[:10]

sorted_10

for item in sorted_10:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Jinx: picked in 7.0120898100172715% of games

Ahri: picked in 5.457685664939551% of games

Graves: picked in 5.2504317789291886% of games

Jhin: picked in 5.112262521588947% of games

Aphelios: picked in 4.6286701208981% of games

Zeri: picked in 4.352331606217617% of games

Gwen: picked in 4.283246977547496% of games

Ezreal: picked in 4.075993091537133% of games

Lee Sin: picked in 3.5233160621761654% of games

Viego: picked in 3.419689119170984% of games



# Least Picked Champions 
##### *With at least one pick*

In [16]:
sorted_10_bottom  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)[:10]

sorted_10_bottom

for item in sorted_10_bottom:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Master Yi: picked in 0.03454231433506045% of games

Anivia: picked in 0.03454231433506045% of games

Kog'Maw: picked in 0.03454231433506045% of games

Rammus: picked in 0.03454231433506045% of games

Warwick: picked in 0.03454231433506045% of games

Nocturne: picked in 0.03454231433506045% of games

Azir: picked in 0.03454231433506045% of games

Annie: picked in 0.0690846286701209% of games

Kled: picked in 0.0690846286701209% of games

Illaoi: picked in 0.0690846286701209% of games



# Champions Not Picked at All

In [17]:
picked_champs = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)

full_champ_list = []

list = cass.get_champions(region='NA')

for item in list:
    full_champ_list.append(item.name)

champions_not_picked = set(full_champ_list).difference(set(picked_champs))


print("These Champions were picked in 0 games:") 
for item in champions_not_picked:
    print('\n' + item)

These Champions were picked in 0 games:

Singed

Taric

Yorick

Skarner

Vel'Koz


# Most one-trick player

In [18]:
# How do I get the mode with my current tools [total_match_list, ('Summoner', 'Champion', 'Match'), summoner_name]
# My current plan is to use a dataframe to get the mode for each Summoner, I can also use the dataframe to get the total matches played per summoner. 

# ^This idea was good in my head but is total dogshit. I think I need to create some function to iterate through the list of tuples, and group them under their summoner name
#  (MAYBE THIS WAS SOMETHING THAT I SHOULD HAVE DONE DURING THE DATA COLLECTION PROCESS BUT HERE WE ARE, WE'RE TRYING TO LEARN EVEN IF ITS NOT OPTIMAL)



# I could then create a list of tuples (Player, Champion, Mode) 
# Not sure what to do afterward

# Perhaps create a parallel list of modes, and use a max function to pull the string, and then use that string to pull the Summoner and Champion index from the tuple

In [30]:
gamer_list = ['taybor', 'dan', 'destiny', 'moot']
tuple_list = [('destiny', 'Tristana', 'Match'), ('moot', 'blitz', 'Match'), ('taybor', 'jinx', 'Match'), ('dan', 'gangplank', 'Match'), ('taybor', 'jinx', 'Match'), ('moot', 'pyke', 'Match')]
picks_aggregated_to_summoner= {0: ['butt'], 1: ['fart'], 2: ['butt'], 3: ['poop'], 4: ['pee']}

for gamer in gamer_list:
    history_list = []
    for tuple in tuple_list:
        if gamer == tuple[0]:
            history_list.append(tuple[1])
    picks_aggregated_to_summoner.update({(int(gamer_list.index(str(gamer)))): history_list})

tuple_list
picks_aggregated_to_summoner

# THIS IS THE PATH IT WORKS!!!!
# now i just have to create a 300 length dictionary



{0: ['jinx', 'jinx'],
 1: ['gangplank'],
 2: ['Tristana'],
 3: ['blitz', 'pyke'],
 4: ['pee']}

# Least one-trick player


# Least Games Played


# Most Games Played
